<a href="https://colab.research.google.com/github/Zimi1214/Final-Project/blob/master/Minimized%20code%20for%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import string

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

seed=4353

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
df=pd.read_csv('/content/drive/My Drive/Data Science/clean.csv')
df

,Unnamed: 0,text,subject,type
0,0,donald trump send embarrass new year eve messa...,News,0
1,1,drunk brag trump staffer start russian collu i...,News,0
2,2,sheriff david clark becom internet joke threat...,News,0
3,3,trump obsess even obama name code websit imag ...,News,0
4,4,pope franci call donald trump christma speechp...,News,0
...,...,...,...,...
44893,21412,fulli commit nato back new us approach afghani...,worldnews,1
44894,21413,lexisnexi withdrew two product chine marketlon...,worldnews,1
44895,21414,minsk cultur hub becom authoritiesminsk reuter...,worldnews,1
44896,21415,vatican upbeat possibl pope franci visit russi...,worldnews,1


In [0]:
df.drop(columns=['Unnamed: 0','subject'],inplace=True,axis=1)

In [0]:
# Function to retrieve processed words

def final(X_data_full):
    
    # function for removing punctuations
    def remove_punct(X_data_func):
        string1 = X_data_func.lower()
        translation_table = dict.fromkeys(map(ord, string.punctuation),' ')
        string2 = string1.translate(translation_table)
        return string2
    
    X_data_full_clear_punct = []
    for i in range(len(X_data_full)):
        test_data = remove_punct(X_data_full[i])
        X_data_full_clear_punct.append(test_data)
        
    # function to remove stopwords
    def remove_stopwords(X_data_func):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        string2 = pattern.sub(' ', X_data_func)
        return string2
    
    X_data_full_clear_stopwords = []
    for i in range(len(X_data_full)):
        test_data = remove_stopwords(X_data_full[i])
        X_data_full_clear_stopwords.append(test_data)
        
    # function for tokenizing
    def tokenize_words(X_data_func):
        words = nltk.word_tokenize(X_data_func)
        return words
    
    X_data_full_tokenized_words = []
    for i in range(len(X_data_full)):
        test_data = tokenize_words(X_data_full[i])
        X_data_full_tokenized_words.append(test_data)
        
    # function for lemmatizing
    lemmatizer = WordNetLemmatizer()
    def lemmatize_words(X_data_func):
        words = lemmatizer.lemmatize(X_data_func)
        return words
    
    X_data_full_lemmatized_words = []
    for i in range(len(X_data_full)):
        test_data = lemmatize_words(X_data_full[i])
        X_data_full_lemmatized_words.append(test_data)
        
    # creating the bag of words model
    cv = CountVectorizer(max_features=1000)
    X_data_full_vector = cv.fit_transform(X_data_full_lemmatized_words).toarray()
    
    
    tfidf = TfidfTransformer()
    X_data_full_tfidf = tfidf.fit_transform(X_data_full_vector).toarray()
    
    return X_data_full_tfidf

In [0]:
# Preparing training and testing data using train_test_split

# Data preparation

X_data = df['text']
y_data = df.type
X_data = X_data.astype(str)
x_data=final(X_data)


In [9]:
# NB fitting and prediction

NBX_train, NBX_test, NBy_train, NBy_test = train_test_split(x_data, y_data, test_size=0.3, random_state= seed)

MNB = MultinomialNB()
MNB.fit(NBX_train, NBy_train)
NB_predictions = MNB.predict(NBX_test)

# NB Model evaluation

print("Classification Report", "\n" ,classification_report(NBy_test, NB_predictions))
print("Confusion Matrix", "\n" ,confusion_matrix(NBy_test, NB_predictions))

MNB_f1 = round(f1_score(NBy_test, NB_predictions, average='weighted'), 3)
MNB_accuracy = round((accuracy_score(NBy_test, NB_predictions)*100),2)

print("\n","NB Evaluation:","\n","Accuracy : " , MNB_accuracy , " %")
print("f1_score : " , MNB_f1)

Classification Report 
               precision    recall  f1-score   support

           0       0.93      0.93      0.93      7057
           1       0.92      0.92      0.92      6413

    accuracy                           0.93     13470
   macro avg       0.93      0.93      0.93     13470
weighted avg       0.93      0.93      0.93     13470

Confusion Matrix 
 [[6568  489]
 [ 507 5906]]

 NB Evaluation: 
 Accuracy :  92.61  %
f1_score :  0.926


In [10]:
# RF fitting and prediction

RFX_train, RFX_test, RFy_train, RFy_test = train_test_split(x_data, y_data, test_size=0.3, random_state= seed)

rfc=RandomForestClassifier(n_estimators= 10, random_state= seed)
rfc.fit(RFX_train, RFy_train)
RF_predictions = rfc.predict(RFX_test)

# RF Model evaluation

print("Classification Report", "\n" , classification_report(RFy_test, RF_predictions))
print("Confusion Matrix", "\n" ,confusion_matrix(RFy_test, RF_predictions))

rfc_f1 = round(f1_score(RFy_test, RF_predictions, average= 'weighted'), 3)
rfc_accuracy = round((accuracy_score(RFy_test, RF_predictions) * 100), 2)

print("\n","RF Evaluation:","\n","Accuracy : " , rfc_accuracy , " %")
print("f1_score : " , rfc_f1)

Classification Report 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      7057
           1       1.00      0.99      0.99      6413

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470

Confusion Matrix 
 [[7027   30]
 [  75 6338]]

 RF Evaluation: 
 Accuracy :  99.22  %
f1_score :  0.992


In [0]:
# Function to predict the type based on input test text

def test_news(X_data_full):
    
    # function for removing punctuations
    def remove_punct(X_data_func):
        string1 = X_data_func.lower()
        translation_table = dict.fromkeys(map(ord, string.punctuation),' ')
        string2 = string1.translate(translation_table)
        return string2
    
    X_data_full_clear_punct = []
    for i in range(len(X_data_full)):
        test_data = remove_punct(X_data_full[i])
        X_data_full_clear_punct.append(test_data)
        
    # function to remove stopwords
    def remove_stopwords(X_data_func):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        string2 = pattern.sub(' ', X_data_func)
        return string2
    
    X_data_full_clear_stopwords = []
    for i in range(len(X_data_full)):
        test_data = remove_stopwords(X_data_full[i])
        X_data_full_clear_stopwords.append(test_data)
        
    # function for tokenizing
    def tokenize_words(X_data_func):
        words = nltk.word_tokenize(X_data_func)
        return words
    
    X_data_full_tokenized_words = []
    for i in range(len(X_data_full)):
        test_data = tokenize_words(X_data_full[i])
        X_data_full_tokenized_words.append(test_data)
        
    # function for lemmatizing
    lemmatizer = WordNetLemmatizer()
    def lemmatize_words(X_data_func):
        words = lemmatizer.lemmatize(X_data_func)
        return words
    
    X_data_full_lemmatized_words = []
    for i in range(len(X_data_full)):
        test_data = lemmatize_words(X_data_full[i])
        X_data_full_lemmatized_words.append(test_data)
    
    # creating the bag of words model
    cv = CountVectorizer(max_features=1000)
    X_data_full_vector = cv.fit_transform(X_data_full_lemmatized_words).toarray()
    
    tfidf = TfidfTransformer()
    X_data_full_tfidf = tfidf.fit_transform(X_data_full_vector).toarray()
    
    return X_data_full_tfidf

In [25]:
h=["break michael flynn crack testifi mueller trump himselfmichael flynn trump embattl former nation secur advi reportedli cave testifi robert mueller team trump collu russia accord abc news special report flynn plead guilti charg includ make fal statement fbi importantli admit plea offici trump transit team direct contact russian officialsfurthermor accord cnn david wright twitter report brian ross report abc news said flynn also say prepar testifi trump order direct make contact russian contradict donald trump said point brianross report michael flynn prepar testifi presid trump candid donald trump order direct make contact russian contradict donald trump said point david wright davidwrightcnn decemb brianross well told flynn made deci cooper last hour distraught deci feel right thing countri david wright davidwrightcnn decemb brianross face huge legal bill million dollar said final go reason expect put hou market face seriou financ problem david wright davidwrightcnn decemb part flynn issu statement say follow action acknowledg court today wrong faith god work set thing right guilti plea agreement cooper special counsel offic reflect deci made best interest famili countri accept full respon action white hou said mere got flynn fire first place zero effect trump har de har har make us laugh hard hurt merri christma donald trump entir treason famili administr hope like orang jumpsuit featur imag via chip somodevillagetti imag"]

h_data=test_news(h)

rfc.predict(h_data)

ValueError: ignored